In [1]:
from datetime import date
import numpy as np
import tensorflow as tf
from tensorflow import keras

2023-12-31 16:43:55.880231: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-31 16:43:55.910294: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-31 16:43:55.910325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-31 16:43:55.910999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-31 16:43:55.915560: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-31 16:43:55.916347: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
MONTHS_DICT = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
              }

In [3]:
def generate_data(n_samples):
    X, y = [], []
    ordinal_min = date(1000, 1, 1).toordinal()
    ordinal_max = date(9999, 12, 31).toordinal()
    
    ordinal_random = np.random.randint(ordinal_max - ordinal_min, size=n_samples) + ordinal_min

    for ordinal in ordinal_random:
        dt = date.fromordinal(ordinal)
        tm_year, tm_month, tm_day = dt.timetuple().tm_year, dt.timetuple().tm_mon, dt.timetuple().tm_mday
        month = MONTHS_DICT[tm_month]
        y.append(dt.isoformat())
        X.append(f"{month} {tm_day}, {tm_year}")
        
    return X,y

In [4]:
INPUT_CHARS = sorted(set("".join(MONTHS_DICT.values()) + "1234567890, "))
OUTPUT_CHARS = "0123456789-"
def vectorize_input(data):
    return [INPUT_CHARS.index(char) for char in data]

def vectorize_output(data):
    return [OUTPUT_CHARS.index(char) for char in data]

In [5]:
def create_dataset(n_samples, batch_size=32):
    x, y = generate_data(n_samples)
    
    X = [vectorize_input(dt) for dt in x]
    Y = [vectorize_output(dt) for dt in y]
    X, Y = tf.ragged.constant(X, ragged_rank=1), tf.ragged.constant(Y,ragged_rank=1)

    X, Y = (X + 1).to_tensor(), (Y + 1).to_tensor()
    
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))
    # dataset = dataset.shuffle(n_samples)
    dataset = dataset.batch(batch_size)
    
    return dataset, X.shape, Y.shape
    


In [6]:
train_dataset, MAX_INPUT_SHAPE, MAX_OUTPUT_SHAPE = create_dataset(n_samples=15000)
test_dataset, _, _ = create_dataset(n_samples=3000)
val_dataset, _, _ = create_dataset(n_samples=2000)

2023-12-31 16:44:03.268445: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-31 16:44:03.283511: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
embedding_size = 32

encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, output_dim=embedding_size, input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")   
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(MAX_OUTPUT_SHAPE[1]),
    decoder
])

In [8]:
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
469/469 [==============================] - 15s 26ms/step - loss: 1.6374 - accuracy: 0.4226 - val_loss: 1.2978 - val_accuracy: 0.5401
Epoch 2/10
469/469 [==============================] - 12s 25ms/step - loss: 0.9816 - accuracy: 0.6429 - val_loss: 0.7724 - val_accuracy: 0.7118
Epoch 3/10
469/469 [==============================] - 12s 25ms/step - loss: 0.6362 - accuracy: 0.7523 - val_loss: 0.5156 - val_accuracy: 0.7918
Epoch 4/10
469/469 [==============================] - 12s 25ms/step - loss: 0.3945 - accuracy: 0.8431 - val_loss: 0.2999 - val_accuracy: 0.8844
Epoch 5/10
469/469 [==============================] - 11s 24ms/step - loss: 0.2856 - accuracy: 0.9055 - val_loss: 0.1735 - val_accuracy: 0.9481
Epoch 6/10
469/469 [==============================] - 11s 24ms/step - loss: 0.1129 - accuracy: 0.9714 - val_loss: 0.0730 - val_accuracy: 0.9859
Epoch 7/10
469/469 [==============================] - 12s 25ms/step - loss: 0.0499 - accuracy: 0.9920 - val_loss: 0.0338 - val_accuracy:

In [9]:
def prepare_input(date_strs):
    ids = [vectorize_input(str) for str in date_strs]
    X = tf.ragged.constant(ids, ragged_rank=1)
    return (X + 1).to_tensor()

In [10]:
X_new = prepare_input(["September 17, 2009", "July 14, 1789"])

In [11]:
ids = np.argmax(model.predict(X_new), axis=-1)

1/1 [==============================] - 0s 439ms/step


In [12]:
for id in ids:
    print(''.join([OUTPUT_CHARS[index -1] for index in id]))

2009-09-17
1789-07-14


In [13]:
X_new = prepare_input(["May 02, 2020", "July 14, 1789"])

In [14]:
ids = np.argmax(model.predict(X_new), axis=-1)
for id in ids:
    print(''.join([OUTPUT_CHARS[index -1] for index in id]))

1/1 [==============================] - 0s 418ms/step
2020-09-21
1789-09-14


In [15]:
X_new = prepare_input(["May 02, 2020", "September 17, 2009"])

In [16]:
X_new[0]

<tf.Tensor: shape=(18,), dtype=int32, numpy=
array([17, 21, 38,  1,  3,  5,  2,  1,  5,  3,  5,  3,  0,  0,  0,  0,  0,
        0], dtype=int32)>

In [17]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [vectorize_input(dt) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor()

In [18]:
def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    print(X)
    if X.shape[1] < MAX_INPUT_SHAPE[1]:
        X = tf.pad(X, [[0, 0], [0, MAX_INPUT_SHAPE[1] - X.shape[1]]])
        print(X)
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = np.argmax(model.predict(X), axis=-1)
    return ids

In [19]:
pred = convert_date_strs(["May 11, 2020", "September 17, 2009"])

tf.Tensor(
[[17 21 38  1  4  4  2  1  5  3  5  3  0  0  0  0  0  0]
 [20 24 32 35 24 29 22 24 33  1  4 10  2  1  5  3  3 12]], shape=(2, 18), dtype=int32)
1/1 [==============================] - 0s 20ms/step


In [20]:
for id in pred:
    print(''.join([OUTPUT_CHARS[index -1] for index in id]))

2020-05-11
2009-09-17


## Teacher Forcing

In [22]:
TOKEN_ID = len(OUTPUT_CHARS) + 1
def shift_output(dataset):
    X = np.concatenate([X for X,Y in dataset], axis=0)
    Y = np.concatenate([Y for X,Y in dataset], axis=0)

    sos_token = tf.fill(dims=(Y.shape[0], 1), value=TOKEN_ID)
    X_decoder = np.concatenate([sos_token, Y[:, :-1]], axis=1)
    
    return tf.constant(X), X_decoder, tf.constant(Y)

In [23]:
X_train, X_train_decoder, y_train = shift_output(train_dataset)
X_test, X_test_decoder, y_test = shift_output(test_dataset)
X_val, X_val_decoder, y_val = shift_output(val_dataset)

February 9, 1912yy
1912-02-09
July 4, 5475yyyyyy
5475-07-04
July 26, 1945yyyyy
1945-07-26
December 7, 4783yy
4783-12-07
September 7, 3262y
3262-09-07
April 28, 4369yyyy
4369-04-28
July 8, 2796yyyyyy
2796-07-08
February 20, 2625y
2625-02-20
October 15, 2898yy
2898-10-15
October 28, 7660yy
7660-10-28
April 24, 8703yyyy
8703-04-24
November 11, 4315y
4315-11-11
February 27, 2078y
2078-02-27
March 7, 3801yyyyy
3801-03-07
January 1, 9897yyy
9897-01-01
August 2, 8515yyyy
8515-08-02
January 30, 5671yy
5671-01-30
August 4, 6677yyyy
6677-08-04
January 12, 8507yy
8507-01-12
July 12, 2042yyyyy
2042-07-12
August 1, 2723yyyy
2723-08-01
May 15, 8726yyyyyy
8726-05-15
January 19, 8776yy
8776-01-19
April 18, 4191yyyy
4191-04-18
August 27, 6071yyy
6071-08-27
December 30, 3992y
3992-12-30
August 7, 9323yyyy
9323-08-07
February 14, 6871y
6871-02-14
April 10, 7881yyyy
7881-04-10
March 25, 1990yyyy
1990-03-25
July 24, 8805yyyyy
8805-07-24
August 31, 6300yyy
6300-08-31
August 13, 2891yyy
2891-08-13
January 26

In [24]:
encoder_input_layer = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding_layer = keras.layers.Embedding(input_dim=len(INPUT_CHARS)+ 1, output_dim=512)(encoder_input_layer)
output, encoder_h_state, encoder_c_state = keras.layers.LSTM(128, return_state=True)(encoder_embedding_layer)

encoder_state = [encoder_h_state, encoder_c_state]

decoder_input_layer = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding_layer = keras.layers.Embedding(input_dim=len(OUTPUT_CHARS)+ 2, output_dim=512)(decoder_input_layer)
decoder_LSTM_output = keras.layers.LSTM(128, return_sequences=True)(decoder_embedding_layer, initial_state=encoder_state)
decoder_output = keras.layers.Dense(len(OUTPUT_CHARS)+1, activation="softmax")(decoder_LSTM_output)

model = keras.models.Model(inputs=[encoder_input_layer, decoder_input_layer], outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], y_train, epochs=20, validation_data=([X_val, X_val_decoder], y_val))

Epoch 1/20
469/469 [==============================] - 20s 37ms/step - loss: 0.7601 - accuracy: 0.7363 - val_loss: 0.2126 - val_accuracy: 0.9431
Epoch 2/20
469/469 [==============================] - 16s 35ms/step - loss: 0.1009 - accuracy: 0.9793 - val_loss: 0.0426 - val_accuracy: 0.9956
Epoch 3/20
469/469 [==============================] - 17s 36ms/step - loss: 0.0280 - accuracy: 0.9974 - val_loss: 0.0164 - val_accuracy: 0.9995
Epoch 4/20
469/469 [==============================] - 16s 35ms/step - loss: 0.0306 - accuracy: 0.9946 - val_loss: 0.0114 - val_accuracy: 0.9998
Epoch 5/20
469/469 [==============================] - 17s 35ms/step - loss: 0.0065 - accuracy: 0.9999 - val_loss: 0.0050 - val_accuracy: 0.9999
Epoch 6/20
469/469 [==============================] - 17s 36ms/step - loss: 0.0070 - accuracy: 0.9990 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 7/20
469/469 [==============================] - 16s 35ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: